In [3]:
import numpy as np
import plotly.graph_objects as go
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

import sys
import os
# 使用当前工作目录
project_root = os.path.dirname(os.getcwd())  # 获取当前目录的父目录
sys.path.insert(0, project_root)
from util.methods import assign_variances  # 若未使用，可保留以兼容后续逻辑

seed = 16
# 定义核函数
kernel = ConstantKernel(25.0) * RBF(length_scale=4.0) + WhiteKernel(1e-3)
gp = GaussianProcessRegressor(kernel=kernel)

# 1) 致密网格：1280 个点
x_dense = np.linspace(1, 200, 1280).reshape(-1, 1)

# 2) 在致密网格上采样一条 GP 曲线（先验）
y_dense = gp.sample_y(x_dense, n_samples=1, random_state=seed).flatten() + 70

# 3) 从 1280 个点中等距选择 128 个点（顺序保持）
# 等距索引：0..1279 中抽 128 个点
select_idx = np.linspace(0, 1279, 128, dtype=int)
x_selected = x_dense[select_idx].flatten()   # 选中的 128 个 x
y_selected = y_dense[select_idx]            # 选中的 128 个 y

# 4) 用核函数在“被选点”上计算协方差（注意：这里用 x_selected 本身，而非整数 1..200）
# 核函数需要二维输入
cov_matrix = kernel(x_selected.reshape(-1, 1))
std_dev = np.sqrt(np.diag(cov_matrix))
corr_matrix = cov_matrix / np.outer(std_dev, std_dev)

# 5) 保存矩阵与均值向量
np.savetxt('Sigma1.txt', cov_matrix, 
           fmt='%.6f',
           header=f'高斯过程协方差矩阵（基于选中128个点）\n核函数: {kernel}')

np.savetxt('correlation_matrix.txt', corr_matrix,
           fmt='%.6f',
           header=f'高斯过程相关性矩阵（基于选中128个点）\n核函数: {kernel}')

np.savetxt('mu.txt', 
           y_selected,
           fmt='%.6f',
           header='选中点上的“真实均值”')

# 6) 分组：按顺序切块为 10 组
group_sizes = [16, 6, 15, 12, 21, 9, 8, 8, 21, 12]
assert sum(group_sizes) == 128, "分组大小之和必须为 128"

group_labels = np.empty(128, dtype=int)
start = 0
for g, size in enumerate(group_sizes):
    end = start + size
    group_labels[start:end] = g
    start = end

group_names = [f"第{i+1}组" for i in range(10)]

# 7) 绘图
fig = go.Figure()

# 连续曲线（基于 1280 个点）
fig.add_trace(go.Scatter(
    x=x_dense.flatten(), 
    y=y_dense,
    mode='lines',
    name='GP曲线',
    line=dict(color='royalblue', width=2)
))

# 颜色：10 组
colors = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
]

# 叠加选中点的分组散点
for group in range(10):
    mask = (group_labels == group)
    fig.add_trace(go.Scatter(
        x=x_selected[mask],
        y=y_selected[mask],
        mode='markers',
        name=group_names[group],
        marker=dict(
            color=colors[group],
            size=8,
            line=dict(width=1, color='DarkSlateGrey')
        )
    ))

fig.update_layout(
    title='高斯过程曲线与分组结果（1280点中等距选取128点，顺序分10组）',
    xaxis_title='x',
    yaxis_title='f(x)',
    template='plotly_white',
    height=1080,
    width=1920,
    legend=dict(
        title='组别',
        x=1.05,
        y=1,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255,255,255,0.8)'
    ),
    margin=dict(r=150) 
)

fig.show()
